In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import imageio
import os
from tqdm import tqdm
import cv2
import scienceplots


plt.style.use(['science', 'ieee'])
plt.rcParams.update({"font.family": "sans", "font.serif": [
                    "cm"], "mathtext.fontset": "cm", "font.size": 22})

In [ ]:
W_list = np.load(f'./data/W_constant.npy')
S_list = np.load(f'./data/S_constant.npy')
time_list = np.load('./data/time_list_constant.npy')
model = "constant"

output_dir = './videos/histogram_videos_fps3'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Determine global min and max for X-axis to ensure consistent plotting
xmin = np.min([np.min(s) for s in S_list]) - 0.1
xmax = np.max([np.max(s) for s in S_list]) + 0.1


filenames = []

for i in tqdm(range(len(S_list))):
    plt.figure(figsize=(12, 6))
    
    sns.histplot(S_list[i].reshape(-1), bins=128, kde=False, color='blue', stat='density', linewidth=0)
    
    mu, std = stats.norm.fit(S_list[i].reshape(-1))
    x = np.linspace(xmin, xmax, 100)
    p = stats.norm.pdf(x, mu, std)
    plt.plot(x, p, 'k', linewidth=1, linestyle='--')
    str_epoch = f'{i}'.zfill(2)
    str_time =  f'{round(float(time_list[i]), 4):0>8.3f}'
    plt.title(f"Epoch:{str_epoch}, Time:{str_time} seconds\nFIT RESULTS: $\mu$ = {mu:.2f}, $\sigma$ = {std:.2f}")
    plt.xlabel('VALUE')
    plt.ylabel('DENSITY')
    plt.grid(True)
    plt.ylim(0, 17)
    
    # Save each frame as a temporary PNG file
    filename = f'{output_dir}/S_{model}_ep_{i}.png'
    plt.savefig(filename)
    plt.close()
    
    filenames.append(filename)


In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'x264' if you have it
fps = 3  # Frames per second

# Determine the resolution of the video
# Read the first image to determine the frame size
first_image_path = filenames[0]
frame = cv2.imread(first_image_path)
height, width, layers = frame.shape
frame_size = (width, height)

# Initialize the video writer
video_filename = f'{output_dir}/S_histograms.mp4'  # Path where the MP4 video will be saved
out = cv2.VideoWriter(video_filename, fourcc, fps, frame_size)

# Loop through all the filenames and write them to the video
for filename in filenames:
    frame = cv2.imread(filename)
    out.write(frame)  # Write out the frame

# Release the video writer
out.release()

# Return the path to the saved video file
video_filename